In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd

In [ ]:
class HousingWebScraper:
    
    def __init__(self):
#         self.wait = new WebDriverWait(self.driver, 10)
        options = webdriver.ChromeOptions()
        #options.add_argument('headless')
        #Initializing the webdriver
        self.driver = webdriver.Chrome(executable_path="chromedriver.exe", options=options) #Change the path to where chromedriver is in your home folder.
        self.wait = WebDriverWait(self.driver, 10)
        self.driver.set_window_size(1120, 1000)

        
    def get_links(self):
        """Get all the houses description links to scrap the item data."""
        self.driver.get('https://housing.com/in/buy/bangalore/house-bangalore')
        total_item = 9000
        a=0
        articles = 0
        time.sleep(2)
        div=0
        try:
            while a < total_item:
                articles = self.driver.find_elements_by_class_name('css-h7k7mr')
                print(len(articles))
                a = len(articles)
                if (len(articles)<total_item):
                    time.sleep(4)
                    self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                else:
                    div = self.driver.find_elements_by_class_name('css-ltqa49')
                    break
        except KeyboardInterrupt:
            div = self.driver.find_elements_by_class_name('css-ltqa49')
        return [ele.find_element_by_css_selector('a').get_attribute('href') for ele in div]

    def get_links_data(self, all_urls):
        """Extracts and returns company links (maximum number of company links for return is provided)."""
#         WebDriverWait wait = new WebDriverWait(self.driver, 10)
        houses = []
        windows_before  = self.driver.current_window_handle # Store the parent_window_handle for future use
        for my_href in all_urls:
            self.driver.execute_script("window.open('" + my_href +"');")
            WebDriverWait(self.driver, 10).until(EC.number_of_windows_to_be(2)) # Induce  WebDriverWait for the number_of_windows_to_be 2
            windows_after = self.driver.window_handles
            new_window = [x for x in windows_after if x != windows_before][0] # Identify the newly opened window
            self.driver.switch_to.window(new_window) # switch_to the new window
            houses.append(self.scrap_newtab_data())
            self.driver.close() # close the window
            self.driver.switch_to.window(windows_before) # switch
        return houses

    def scrap_newtab_data(self):
        """Extracts and House specific data from the new tab."""
        data = {}
        WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,'css-r74jsk')))
            
        try:
            section_att = self.driver.find_elements_by_class_name('css-r74jsk')
            section = self.driver.find_elements_by_class_name('css-3o6ku8')
            tempDict = {}
            for a,v in zip(section_att,section):
                tempDict[a.text]=v.text
                
            section2_att = self.driver.find_elements_by_class_name('css-14kcwq8')
            section2 = self.driver.find_elements_by_class_name('css-19ug521')
            temp2 = {}
            for a,v in zip(section2_att,section2):
                temp2[a.text]=v.text

            try:
                bhk = self.driver.find_element_by_class_name('css-10rvbm3').text
            except NoSuchElementException:
                bhk = -1

            try:
                address = self.driver.find_element_by_class_name('css-1ru8tnn').text
            except NoSuchElementException:
                address = -1

            try:
                persqrft = self.driver.find_element_by_class_name('css-v161dt').text
            except NoSuchElementException:
                persqrft = -1
                
        except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
            pass
        data ={"BHK" : bhk,
            "ADDRESS" : address,
            "PERSQRFT" : persqrft}
        data.update(tempDict)
        data.update(temp2)
        print(data)
        return data
#     return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [ ]:
scraper = HousingWebScraper()
houselinks = scraper.get_links()
houses = scraper.get_links_data(houselinks)
df = pd.DataFrame(houses)

In [ ]:
df.to_csv('housing_data.csv')